In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import LinearIVGMM
import matplotlib.pyplot as plt

In [2]:
# Sensitivity analysis using actual inflation at time t

# Get CPI index data
data_cpi = pd.read_parquet("data_inf.parquet")

# OLS
data_cpi1 = data_cpi.dropna(subset=['output_gap_plucking'])
X = sm.add_constant(data_cpi1[['inf_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_plucking']]

data_cpi2 = data_cpi.dropna(subset=['output_gap_bb_avg'])
X = sm.add_constant(data_cpi2[['inf_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_bb_avg']]


data_cpi3 = data_cpi.dropna(subset=['output_gap_rgdp_yoy'])
X = sm.add_constant(data_cpi3[['inf_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_dev'], results.pvalues['output_gap_rgdp_yoy']]



# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf'], 2)[2:]
constant = np.ones_like(endog)[2:] 
endog = endog[2:] 
exog = exog[2:] 
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant)) 
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf'], 2)[2:]
constant = np.ones_like(endog)[2:]  
endog = endog[2:]  
exog = exog[2:]  
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [3]:
# Sensitivity analysis using inf3ma

# OLS
X = sm.add_constant(data_cpi1[['inf_3ma_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_3ma_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf3ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf3ma_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_plucking']]

X = sm.add_constant(data_cpi2[['inf_3ma_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_3ma_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf3ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf3ma_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_bb_avg']]

X = sm.add_constant(data_cpi3[['inf_3ma_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_3ma_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf3ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf3ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf3ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf3ma_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_3ma_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf3ma_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_3ma_dev'], results.pvalues['output_gap_rgdp_yoy']]


# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_3ma_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf3ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_3ma_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf3ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_3ma_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf_3ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf_3ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf3ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf3ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf3ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf3ma_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf3ma_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [4]:
# Sensitivity analysis using inf5ma

# OLS
X = sm.add_constant(data_cpi1[['inf_5ma_dev', 'output_gap_plucking', 'opr_lag']])
model = sm.OLS(data_cpi1['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_5ma_dev']
coef_output_gap = coefficients['output_gap_plucking']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf5ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_plucking['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_plucking'] / (1 - alpha)]
results_table_ols_inf5ma_plucking['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_plucking']]

X = sm.add_constant(data_cpi2[['inf_5ma_dev', 'output_gap_bb_avg', 'opr_lag']])
model = sm.OLS(data_cpi2['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_inf_dev = coefficients['inf_5ma_dev']
coef_output_gap = coefficients['output_gap_bb_avg']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_inf_dev / (1 - alpha)
gamma = coef_output_gap / (1 - alpha)
results_table_ols_inf5ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_bb['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_bb_avg'] / (1 - alpha)]
results_table_ols_inf5ma_bb['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_bb_avg']]

X = sm.add_constant(data_cpi3[['inf_5ma_dev', 'output_gap_rgdp_yoy', 'opr_lag']])
model = sm.OLS(data_cpi3['opr'], exog=X)
results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
coefficients = results.params
coef_const = coefficients['const']
coef_infx_analyst_dev = coefficients['inf_5ma_dev']
coef_output_rgdp_yoy = coefficients['output_gap_rgdp_yoy']
coef_opr_lag = coefficients['opr_lag']
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_rgdp_yoy / (1 - alpha)
results_table_ols_inf5ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_ols_inf5ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_ols_inf5ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_ols_inf5ma_rgdp['Standard Error'] = [results.bse['opr_lag'], results.bse['const'] / (1 - alpha), results.bse['inf_5ma_dev'] / (1 - alpha), results.bse['output_gap_rgdp_yoy'] / (1 - alpha)]
results_table_ols_inf5ma_rgdp['P-value'] = [results.pvalues['opr_lag'], results.pvalues['const'], results.pvalues['inf_5ma_dev'], results.pvalues['output_gap_rgdp_yoy']]

# GMM
exog = np.column_stack((np.ones_like(data_cpi1['opr_lag']), data_cpi1['inf_5ma_dev'], data_cpi1['output_gap_plucking'], data_cpi1['opr_lag']))
endog = data_cpi1['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi1['output_gap_plucking'], 1)[2:]
instruments_2 = np.roll(data_cpi1['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi1['output_gap_plucking'], 2)[2:]
instruments_5 = np.roll(data_cpi1['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_plucking = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_plucking / (1 - alpha)
results_table_gmm_inf5ma_plucking = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_plucking['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_plucking['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_plucking['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_plucking['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi2['opr_lag']), data_cpi2['inf_5ma_dev'], data_cpi2['output_gap_bb_avg'], data_cpi2['opr_lag']))
endog = data_cpi2['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi2['output_gap_bb_avg'], 1)[2:]
instruments_2 = np.roll(data_cpi2['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi2['output_gap_bb_avg'], 2)[2:]
instruments_5 = np.roll(data_cpi2['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_bb_avg = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_bb_avg / (1 - alpha)
results_table_gmm_inf5ma_bb = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_bb['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_bb['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_bb['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_bb['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

exog = np.column_stack((np.ones_like(data_cpi3['opr_lag']), data_cpi3['inf_5ma_dev'], data_cpi3['output_gap_rgdp_yoy'], data_cpi3['opr_lag']))
endog = data_cpi3['opr']
instruments = np.roll(endog, 1)[2:]  # Lagged dependent variable as an instrument
instruments_1 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 1)[2:]
instruments_2 = np.roll(data_cpi3['inf_5ma'], 1)[2:]
instruments_3 = np.roll(endog, 2)[2:]  # Lagged dependent variable as an instrument
instruments_4 = np.roll(data_cpi3['output_gap_rgdp_yoy'], 2)[2:]
instruments_5 = np.roll(data_cpi3['inf_5ma'], 2)[2:]
constant = np.ones_like(endog)[2:]  # Constant as an instrument
endog = endog[2:]  # Remove the first row to match the length of instruments
exog = exog[2:]  # Remove the first row to match the length of instruments
instruments = np.column_stack((instruments, instruments_1, instruments_2, instruments_3, instruments_4, instruments_5 ,constant))  # Combine instruments
gmm_model = LinearIVGMM(endog, exog, instrument=instruments)
gmm_results = gmm_model.fit()
coefficients = gmm_results.params
coef_const = coefficients[0]
coef_infx_analyst_dev = coefficients[1]
coef_output_gap_rgdp_yoy = coefficients[2]
coef_opr_lag = coefficients[3]
alpha = coef_opr_lag
r = coef_const / (1 - alpha)
beta = coef_infx_analyst_dev / (1 - alpha)
gamma = coef_output_gap_rgdp_yoy / (1 - alpha)
results_table_gmm_inf5ma_rgdp = pd.DataFrame(columns=['Coefficient', 'Estimated Value', 'Standard Error', 'P-value'])
results_table_gmm_inf5ma_rgdp['Coefficient'] = ['alpha', 'r', 'beta', 'gamma']
results_table_gmm_inf5ma_rgdp['Estimated Value'] = [alpha, r, beta, gamma]
results_table_gmm_inf5ma_rgdp['Standard Error'] = [gmm_results.bse[3], gmm_results.bse[0] / (1 - alpha), gmm_results.bse[1] / (1 - alpha), gmm_results.bse[2] / (1 - alpha)]
results_table_gmm_inf5ma_rgdp['P-value'] = [gmm_results.pvalues[3], gmm_results.pvalues[0], gmm_results.pvalues[1], gmm_results.pvalues[2]]

In [5]:
# Table 3
# Consolidate sensitivity analysis results

sensitivity_results_table = pd.concat([ results_table_ols_inf_plucking, results_table_gmm_inf_plucking, 
                                   results_table_ols_inf3ma_plucking, results_table_gmm_inf3ma_plucking, 
                                   results_table_ols_inf5ma_plucking, results_table_gmm_inf5ma_plucking, 
                                   results_table_ols_inf_bb, results_table_gmm_inf_bb, 
                                   results_table_ols_inf3ma_bb, results_table_gmm_inf3ma_bb, 
                                   results_table_ols_inf5ma_bb, results_table_gmm_inf5ma_bb, 
                                   results_table_ols_inf_rgdp, results_table_gmm_inf_rgdp, 
                                   results_table_ols_inf3ma_rgdp, results_table_gmm_inf3ma_rgdp, 
                                   results_table_ols_inf5ma_rgdp, results_table_gmm_inf5ma_rgdp], 
                                  ignore_index=True)

# Create labels for the models
ols_inf_plucking_label = 'OLS (Inf - Plucking)'
gmm_inf_plucking_label = 'GMM (Inf - Plucking)'
ols_inf3ma_plucking_label = 'OLS (Inf3ma - Plucking)'
gmm_inf3ma_plucking_label = 'GMM (Inf3ma - Plucking)'
ols_inf5ma_plucking_label = 'OLS (Inf5ma - Plucking)'
gmm_inf5ma_plucking_label = 'GMM (Inf5ma - Plucking)'
ols_inf_bb_label = 'OLS (Inf - BB)'
gmm_inf_bb_label = 'GMM (Inf - BB)'
ols_inf3ma_bb_label = 'OLS (Inf3ma - BB)'
gmm_inf3ma_bb_label = 'GMM (Inf3ma - BB)'
ols_inf5ma_bb_label = 'OLS (Inf5ma - BB)'
gmm_inf5ma_bb_label = 'GMM (Inf5ma - BB)'
ols_inf_rgdp_label = 'OLS (Inf - RGDP)'
gmm_inf_rgdp_label = 'GMM (Inf - RGDP)'
ols_inf3ma_rgdp_label = 'OLS (Inf3ma - RGDP)'
gmm_inf3ma_rgdp_label = 'GMM (Inf3ma - RGDP)'
ols_inf5ma_rgdp_label = 'OLS (Inf5ma - RGDP)'
gmm_inf5ma_rgdp_label = 'GMM (Inf5ma - RGDP)'

# Create a new column for model labels
model_labels = ([ols_inf_plucking_label] * len(results_table_ols_inf_plucking) + 
               [gmm_inf_plucking_label] * len(results_table_gmm_inf_plucking) + 
               [ols_inf3ma_plucking_label] * len(results_table_ols_inf3ma_plucking) + 
               [gmm_inf3ma_plucking_label] * len(results_table_gmm_inf3ma_plucking) + 
               [ols_inf5ma_plucking_label] * len(results_table_ols_inf5ma_plucking) + 
               [gmm_inf5ma_plucking_label] * len(results_table_gmm_inf5ma_plucking) + 
               [ols_inf_bb_label] * len(results_table_ols_inf_bb) + 
               [gmm_inf_bb_label] * len(results_table_gmm_inf_bb) + 
               [ols_inf3ma_bb_label] * len(results_table_ols_inf3ma_bb) + 
               [gmm_inf3ma_bb_label] * len(results_table_gmm_inf3ma_bb) + 
               [ols_inf5ma_bb_label] * len(results_table_ols_inf5ma_bb) + 
               [gmm_inf5ma_bb_label] * len(results_table_gmm_inf5ma_bb) + 
               [ols_inf_rgdp_label] * len(results_table_ols_inf_rgdp) + 
               [gmm_inf_rgdp_label] * len(results_table_gmm_inf_rgdp) + 
               [ols_inf3ma_rgdp_label] * len(results_table_ols_inf3ma_rgdp) + 
               [gmm_inf3ma_rgdp_label] * len(results_table_gmm_inf3ma_rgdp) + 
               [ols_inf5ma_rgdp_label] * len(results_table_ols_inf5ma_rgdp) + 
               [gmm_inf5ma_rgdp_label] * len(results_table_gmm_inf5ma_rgdp))

# Add the new column to the merged results table
sensitivity_results_table.insert(0, 'Model', model_labels)

# Add a row to showcase the significance based on p-values
sensitivity_results_table['Significance'] = sensitivity_results_table['P-value'].apply(lambda p: '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.1 else '')


# Round the values in the "Estimated Value" column to 3 significant figures
sensitivity_results_table["Estimated Value"] = sensitivity_results_table["Estimated Value"].round(3)
sensitivity_results_table["Standard Error"] = sensitivity_results_table["Standard Error"].round(3)

for i, row in sensitivity_results_table.iterrows():
    sensitivity_results_table.at[i, "Estimated Value"] = f"{row['Estimated Value']} {row['Significance']}"
    sensitivity_results_table.at[i, "Significance"] = ''

# Drop the "Significance" and "P-value" columns
sensitivity_results_table.drop(columns=["Significance", "P-value"], inplace=True)

# Iterate over each row of the sensitivity_results_table
for index, row in sensitivity_results_table.iterrows():
    # Get the Model name
    model = row["Model"]
    # Iterate over the columns and find the corresponding Standard Error
    for col in sensitivity_results_table.columns:
        if col.endswith("Standard Error"):
            standard_error_col = col
            break
    # Update the Estimated Value with Standard Error
    sensitivity_results_table.at[index, "Estimated Value"] += " (" + str(row[standard_error_col]) + ")"

# Drop the Standard Error columns
sensitivity_results_table.drop(columns=sensitivity_results_table.filter(like="Standard Error").columns, inplace=True)

# Pivot the table
sensitivity_results_table = sensitivity_results_table.pivot(index="Coefficient", columns="Model", values=["Estimated Value"])

# Print the merged results table
sensitivity_results_table


Estimated Value                                          \
Model           GMM (Inf - BB) GMM (Inf - Plucking)   GMM (Inf - RGDP)   
Coefficient                                                              
alpha        0.892 *** (0.098)    0.958 *** (0.034)  0.949 *** (0.037)   
beta            0.213  (0.144)       0.466 * (0.25)     0.113  (0.203)   
gamma            0.06  (0.126)      -0.001  (0.059)   0.568 *** (0.15)   
r               2.903  (2.506)       3.003  (2.319)     2.832  (2.195)   

                                                                            \
Model        GMM (Inf3ma - BB) GMM (Inf3ma - Plucking) GMM (Inf3ma - RGDP)   
Coefficient                                                                  
alpha        0.993 *** (0.078)       0.953 *** (0.034)   0.911 *** (0.052)   
beta             -0.2  (2.926)          -0.134  (0.31)       0.52  (0.428)   
gamma            -0.6  (1.871)         -0.016  (0.058)   0.334 *** (0.088)   
r              4.732  (31.808)          3.003  (1.899)     2.822 * (1.691)   

                                                                            \
Model        GMM (Inf5ma - BB) GMM (Inf5ma - Plucking) GMM (Inf5ma - RGDP)   
Coefficient                                                                  
alpha        0.949 *** (0.059)       0.928 *** (0.032)   0.926 *** (0.037)   
beta           -0.292  (0.364)         -0.312  (0.231)      0.745  (0.682)   
gamma           -0.01  (0.173)         -0.028  (0.032)   0.472 *** (0.125)   
r               3.082  (3.319)        2.944 ** (1.271)     2.685 * (1.464)   

                                                                        \
Model           OLS (Inf - BB) OLS (Inf - Plucking)   OLS (Inf - RGDP)   
Coefficient                                                              
alpha        0.812 *** (0.067)    0.921 *** (0.039)  0.909 *** (0.037)   
beta          0.141 *** (0.05)      0.312 * (0.174)     0.245  (0.202)   
gamma          0.122 ** (0.05)       0.045  (0.034)     0.187  (0.143)   
r             2.762 *** (0.98)      2.95 ** (1.408)   2.611 ** (1.171)   

                                                                            \
Model        OLS (Inf3ma - BB) OLS (Inf3ma - Plucking) OLS (Inf3ma - RGDP)   
Coefficient                                                                  
alpha         0.839 *** (0.06)       0.939 *** (0.036)   0.904 *** (0.056)   
beta           -0.045  (0.162)           0.06  (0.429)      0.255  (0.704)   
gamma         0.142 ** (0.062)          0.066  (0.049)     0.221 * (0.132)   
r            2.827 *** (1.044)         3.151 * (1.689)      2.604  (1.722)   

                                                                           
Model       OLS (Inf5ma - BB) OLS (Inf5ma - Plucking) OLS (Inf5ma - RGDP)  
Coefficient                                                                
alpha        0.84 *** (0.065)        0.945 *** (0.04)   0.929 *** (0.037)  
beta          -0.164  (0.157)         -0.319  (0.427)     -0.052  (1.083)  
gamma        0.144 ** (0.061)          0.068  (0.054)       0.29  (0.181)  
r            2.888 ** (1.139)          3.336  (2.208)     2.595 * (1.525)